<a href="https://colab.research.google.com/github/castorhyun/DeepLearningZeroToAll/blob/master/dacon_biLSTM_20200613.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://newfront.dacon.io/competitions/official/235401/codeshare/661
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 23), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [0]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [0]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 3.6MB 54.2MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-06-13 05:46:50--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.0, 18.205.93.2, 18.205.93.1, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.0|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive/Colab Notebooks/0001. KB_smishingData/Data" #현재 drive 경로 안의 파일 or 폴더 표시

In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/0001. KB_smishingData/Data

/content/gdrive/My Drive/Colab Notebooks/0001. KB_smishingData/Data


data load


In [0]:
import pandas as pd
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("public_test.csv")

Train


In [0]:
from tqdm import tqdm
train_data_text = list(train_data['text'])

train_clear_text = []

for i in tqdm(range(len(train_data_text))):
  train_clear_text.append(train_data_text[i].replace('XXX', ''))
train_data['clear_text'] = train_clear_text

100%|██████████| 295945/295945 [00:00<00:00, 676094.96it/s]


In [0]:
import re

train_clear_text = list(train_data['clear_text'])

train_clear_text2 = []

for text in train_clear_text:
  temp = re.sub('[-=+,#/\?:^$.@*\"※~>`\'…》]', '', text)
  train_clear_text2.append(temp)
train_data['clear_text2'] = train_clear_text2

In [0]:
clear_text2 = list(train_data['clear_text2'])

text_lens = []

for i in tqdm(range(len(clear_text2))):
  text_lens.append(len(clear_text2[i]))

train_data['len'] = text_lens

100%|██████████| 295945/295945 [00:00<00:00, 1738156.07it/s]


In [0]:
normal_df = train_data[train_data['smishing'] == 0]
normal_df = normal_df.sort_values(by=['len'], axis=0, ascending=False)

normal_df = normal_df.iloc[0:2000]

In [0]:
normal_df.head(10)

,id,year_month,text,smishing,clear_text,clear_text2,len
31832,37293,2017-04,XXX 고객님(44 마감 기준 시황입니다.)- KOSPI: 2161.10pt(-0....,0,고객님(44 마감 기준 시황입니다.)- KOSPI: 2161.10pt(-0.30%...,고객님(44 마감 기준 시황입니다) KOSPI 216110pt(030%) KOS...,1177
192738,219895,2018-01,주요 증시 전거래일 기준 KOSPI 2503.73 7.31 0....,0,주요 증시 전거래일 기준 KOSPI 2503.73 7.31 0....,주요 증시 전거래일 기준 KOSPI 250373 731 029%...,1171
16999,19897,2017-02,XXX목동XXX XXX팀장입니다(215 마감 기준)- KOSPI: 2083.86pt...,0,목동 팀장입니다(215 마감 기준)- KOSPI: 2083.86pt(0.45%) -...,목동 팀장입니다(215 마감 기준) KOSPI 208386pt(045%) KOSP...,1165
195135,222447,2018-01,안녕하세요? XXX 고객님주요 증시 전거래일 기준 KOSPI ...,0,안녕하세요? 고객님주요 증시 전거래일 기준 KOSPI 2...,안녕하세요 고객님주요 증시 전거래일 기준 KOSPI 25...,1165
27326,32018,2017-03,XXX목동XXX XXX팀장입니다(321 마감 기준)- KOSPI: 2178.38pt...,0,목동 팀장입니다(321 마감 기준)- KOSPI: 2178.38pt(0.99%) -...,목동 팀장입니다(321 마감 기준) KOSPI 217838pt(099%) KOSP...,1164
67471,78782,2017-05,XXX 목동XXX XXX팀장입니다(55 마감 기준)한국(코스피) 2241.24 21...,0,목동 팀장입니다(55 마감 기준)한국(코스피) 2241.24 21.57 (0.97...,목동 팀장입니다(55 마감 기준)한국(코스피) 224124 2157 (097%) ...,1161
15277,17779,2017-02,(0131 마감 기준)- KOSPI: 2067.57pt(-0.77%)- KOSPI2...,0,(0131 마감 기준)- KOSPI: 2067.57pt(-0.77%)- KOSPI2...,(0131 마감 기준) KOSPI 206757pt(077%) KOSPI200 268...,1154
259399,294349,2018-08,XXX 고객님안녕하십니까? 주요증시전거래일 기준 KOSPI 22...,0,고객님안녕하십니까? 주요증시전거래일 기준 KOSPI 2247....,고객님안녕하십니까 주요증시전거래일 기준 KOSPI 224788...,1148
109907,127775,2017-06,XXX 목동XXX XXX팀장입니다주요 증시 전거래일 기준 KOSPI ...,0,목동 팀장입니다주요 증시 전거래일 기준 KOSPI 2391.95 ...,목동 팀장입니다주요 증시 전거래일 기준 KOSPI 239195 ...,1144
258037,292830,2018-08,XXX 고객님안녕하십니까? 주요증시전거래일 기준 KOSPI 휴장...,0,고객님안녕하십니까? 주요증시전거래일 기준 KOSPI 휴장 K...,고객님안녕하십니까 주요증시전거래일 기준 KOSPI 휴장 KO...,1142


In [0]:
smishing_df = train_data[train_data['smishing'] == 1]

smishing_df = smishing_df.iloc[0:2000]

In [0]:
new_train_data = pd.concat([normal_df, smishing_df])
new_train_data = new_train_data.sort_values(by=['id'], axis=0, ascending=True)
del new_train_data['len']

학습모델

In [0]:
from konlpy.tag import Okt
from konlpy.tag import Mecab
okt = Okt()
mecab = Mecab()

In [0]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [0]:
# Train 데이터 토큰화
from tqdm import tqdm
X_train = []

text = list(new_train_data['clear_text2'])

for i in tqdm(range(len(text))):
  temp_X = []
  temp_X = okt.morphs(text[i], stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  temp_X = [word for word in temp_X if len(word) > 1]
  X_train.append(temp_X)

100%|██████████| 4000/4000 [04:31<00:00, 14.73it/s]


In [0]:
# 학습데이터 Label
from keras.utils import np_utils
import numpy as np

y_train = []

for i in range(len(new_train_data['smishing'])):
  if new_train_data['smishing'].iloc[i] == 1:
    y_train.append(1)
  elif new_train_data['smishing'].iloc[i] == 0:
    y_train.append(0)

y_train = np.array(y_train)

Keras 인코딩

In [0]:
from keras.preprocessing.text import Tokenizer
max_words = 35000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)


모델학습하기

In [0]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import f1_score 


In [0]:
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [0]:
max_len = 315 # 전체 데이터의 길이를 315로 맞춘다

X_train = pad_sequences(X_train, maxlen=max_len)


In [0]:
model_biLSTM = Sequential()
model_biLSTM.add(Embedding(max_words, 100))
model_biLSTM.add( Bidirectional(LSTM(128)) )
model_biLSTM.add(Dense(1, activation='sigmoid'))

model_biLSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precision, recall, f1score])
history3 = model_biLSTM.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3200 samples, validate on 800 samples
Epoch 1/5
3200/3200 [==============================] - 84s 26ms/step - loss: 0.1634 - accuracy: 0.9287 - precision: 0.9276 - recall: 0.9832 - f1score: 0.9482 - val_loss: 0.0523 - val_accuracy: 0.9937 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 2/5
3200/3200 [==============================] - 82s 26ms/step - loss: 0.0126 - accuracy: 0.9978 - precision: 0.9975 - recall: 0.9989 - f1score: 0.9982 - val_loss: 0.0053 - val_accuracy: 0.9962 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 3/5
3200/3200 [==============================] - 82s 26ms/step - loss: 0.0053 - accuracy: 0.9994 - precision: 0.9994 - recall: 0.9994 - f1score: 0.9994 - val_loss: 0.0078 - val_accuracy: 0.9975 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 4/5
3200/3200 [==============================] - 82s 26ms/step - loss: 0.0012 - accuracy: 1.0000 - precision: 

In [0]:
model_LSTM = Sequential()
model_LSTM.add(Embedding(max_words, 100))
model_LSTM.add(LSTM(128) )
model_LSTM.add(Dense(1, activation='sigmoid'))

model_LSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', precision, recall, f1score])
history3 = model_LSTM.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1600 samples, validate on 400 samples
Epoch 1/3
1600/1600 [==============================] - 24s 15ms/step - loss: 0.3254 - accuracy: 0.9125 - precision: 0.9082 - recall: 0.9901 - f1score: 0.9398 - val_loss: 0.0964 - val_accuracy: 0.9725 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 2/3
1600/1600 [==============================] - 22s 14ms/step - loss: 0.0244 - accuracy: 0.9956 - precision: 0.9973 - recall: 0.9957 - f1score: 0.9964 - val_loss: 0.0246 - val_accuracy: 0.9925 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00
Epoch 3/3
1600/1600 [==============================] - 22s 14ms/step - loss: 0.0071 - accuracy: 0.9987 - precision: 0.9990 - recall: 0.9987 - f1score: 0.9988 - val_loss: 0.0110 - val_accuracy: 0.9975 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1score: 0.0000e+00


Bi LSTM 모델 학습

In [0]:
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=embedding_size,
                        input_length=maxlen, mask_zero=True))
    model.add(Bidirectional(LSTM(hidden_size, return_sequences=True)))
    model.add(TimeDistributed(Dense(out_size)))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam')

테스트 데이터 예측모델

In [0]:
normal_df = train_data[train_data['smishing'] == 0]
normal_df = normal_df.sort_values(by=['len'], axis=0, ascending=False)

normal_df = normal_df.iloc[2000:4000]

In [0]:
smishing_df = train_data[train_data['smishing'] == 1]

smishing_df = smishing_df.iloc[2000:4000]

In [0]:
test_data = pd.concat([normal_df, smishing_df])
test_data = test_data.sort_values(by=['id'], axis=0, ascending=True)
del test_data['len']

In [0]:
test_data.head(1)

,id,year_month,text,smishing,clear_text,clear_text2
20,23,2017-01,시장동향(0130 마감 기준)-KOSPI: 2083.59pt(0.81%)126 마감...,0,시장동향(0130 마감 기준)-KOSPI: 2083.59pt(0.81%)126 마감...,시장동향(0130 마감 기준)KOSPI 208359pt(081%)126 마감 기준 ...


In [0]:
test_data.to_csv('./biLSTM_4000_test_4th.csv')

In [0]:
# Train 데이터 토큰화
from tqdm import tqdm
X_test = []

text = list(test_data['clear_text2'])

for i in tqdm(range(len(text))):
  temp_X = []
  temp_X = okt.morphs(text[i], stem=True) # 형태소 분리
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  temp_X = [word for word in temp_X if len(word) > 1]
  X_test.append(temp_X)

100%|██████████| 4000/4000 [03:40<00:00, 18.15it/s]


In [0]:
#from keras.preprocessing.text import Tokenizer
#max_words = 35000
#tokenizer = Tokenizer(num_words = max_words)
#tokenizer.fit_on_texts(X_train) # 인코딩 fit은 학습데이터를 기준으로 맞춰줘야 함.

# 토크나이져는 위쪽 학습데이터에서 선언됨.

X_test = tokenizer.texts_to_sequences(X_test)

In [0]:
X_test = pad_sequences(X_test, maxlen=max_len)

In [0]:
mypredict = model_biLSTM.predict(X_test)

In [0]:
mypredict

array([[0.00041217],
       [0.00035363],
       [0.00074172],
       ...,
       [0.00043136],
       [0.00042951],
       [0.00032178]], dtype=float32)

In [0]:
mypredict = model_biLSTM.predict(X_test)
submission_ids = list(test_data['id'])
my_result = []

for i in range(len(mypredict)):
  my_result.append(mypredict[i])

sub_dict = {"id":submission_ids, "smishing":my_result}
submission_df = pd.DataFrame(sub_dict)
submission_df.to_csv("biLSTM_4000_predict_4th.csv", index=False, encoding='utf-8')


In [0]:
public_data = pd.read_csv("public_test.csv")

In [0]:
public_data.head(2)

,id,year_month,text
0,336386,2019-01,즐거운 설날을 맞이하여 건강과 함께 평온한 가정이 되시길바랍니다.XXX흥덕 XXX
1,336396,2019-01,XXX 고객님대여금고만XXX입니다. 연장신청바랍니다 XXX은행고잔지점
